In [6]:
import pandas as pd
from sqlalchemy import create_engine

# -------------------
# MySQL connection
# -------------------
engine = create_engine("mysql+pymysql://root:@localhost/etl_demo")

# -------------------
# 1. EXTRACT
# -------------------
df = pd.read_sql("SELECT * FROM employees", engine)
print("Raw data:")
print(df)

# -------------------
# 2. TRANSFORM
# -------------------
df["age"] = df["age"].fillna(df["age"].mean())
df["salary_after_tax"] = df["salary"] * 0.9

# -------------------
# 3. LOAD (REPLACE TABLE)
# -------------------
df.to_sql(
    name="employees_processed",
    con=engine,
    if_exists="replace",  # DROP + CREATE
    index=False
)

print("\nTable replaced and updated in MySQL.")

# -------------------
# 4. SHOW DATA AFTER REPLACEMENT
# -------------------
df_new = pd.read_sql("SELECT * FROM employees_processed", engine)

print("\nData in MySQL AFTER ETL:")
print(df_new)


Raw data:
   id     name   age  salary
0   1    Alice  30.0   50000
1   2      Bob   NaN   60000
2   3  Charlie  25.0   55000

Table replaced and updated in MySQL.

Data in MySQL AFTER ETL:
   id     name   age  salary  salary_after_tax
0   1    Alice  30.0   50000           45000.0
1   2      Bob  27.5   60000           54000.0
2   3  Charlie  25.0   55000           49500.0


In [1]:
import pandas as pd
from sqlalchemy import create_engine

# -------------------
# 0. MySQL connection (XAMPP)
# -------------------
# Default XAMPP MySQL:
#   user: root
#   password: (empty)
#   host: localhost
#   db: etl_demo  (create this in phpMyAdmin if not already there)
engine = create_engine("mysql+pymysql://root:@localhost/etl_demo")

# -------------------
# 1. EXTRACT (from CSV)
# -------------------
df = pd.read_csv("employees_1000.csv", parse_dates=["hire_date"])
print("Raw CSV data (head):")
print(df.head())

# -------------------
# 2. TRANSFORM
# -------------------
# Handle missing ages and salaries (fill with mean)
if df["age"].isna().any():
    df["age"] = df["age"].fillna(df["age"].mean())

if df["salary"].isna().any():
    df["salary"] = df["salary"].fillna(df["salary"].mean())

# Fill missing city/department with placeholder
df["city"] = df["city"].fillna("Unknown")
df["department"] = df["department"].fillna("Unknown")

# Fill missing hire_date with a default date (or you could drop them)
df["hire_date"] = df["hire_date"].fillna(pd.to_datetime("2015-01-01"))

# Derive a new column: years_of_service
today = pd.to_datetime("2025-01-05")  # or pd.Timestamp.today()
df["years_of_service"] = (today - df["hire_date"]).dt.days / 365.25

print("\nTransformed data (head):")
print(df.head())

# -------------------
# 3. LOAD (to MySQL)
# -------------------
df.to_sql(
    name="employees_extended",
    con=engine,
    if_exists="replace",  # Replace table if exists
    index=False
)

print("\nData successfully loaded into MySQL (employees_extended).")

# -------------------
# 4. VERIFY (read back from MySQL)
# -------------------
df_mysql = pd.read_sql("SELECT * FROM employees_extended LIMIT 10", engine)
print("\nSample from MySQL:")
print(df_mysql)


Raw CSV data (head):
      name   age   salary    city department  hire_date
0  Person1  58.0  74327.0  Bergen        NaN 2017-08-15
1  Person2  48.0  98904.0  Tromsø    Support 2019-12-24
2  Person3  34.0  33797.0    Oslo  Marketing 2024-05-08
3  Person4  27.0  77882.0  Bergen      Sales 2016-11-05
4  Person5  40.0  43718.0     NaN        NaN 2019-12-25

Transformed data (head):
      name   age   salary     city department  hire_date  years_of_service
0  Person1  58.0  74327.0   Bergen    Unknown 2017-08-15          7.392197
1  Person2  48.0  98904.0   Tromsø    Support 2019-12-24          5.034908
2  Person3  34.0  33797.0     Oslo  Marketing 2024-05-08          0.662560
3  Person4  27.0  77882.0   Bergen      Sales 2016-11-05          8.167009
4  Person5  40.0  43718.0  Unknown    Unknown 2019-12-25          5.032170

Data successfully loaded into MySQL (employees_extended).

Sample from MySQL:
       name        age        salary          city department  hire_date  \
0   Person1 